In [40]:
import requests

In [41]:
from bs4 import BeautifulSoup

In [89]:
import selenium
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("", options=options)

/home/mihir/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [1]:
def find_cost_1mg(medicine):
    URL = "https://www.1mg.com/search/all?name="+medicine
    r = requests.get(URL)
    soup = BeautifulSoup(r.content,'html5lib')
#     print(soup.prettify())
    name = ""
    cost = ""
    name_table = soup.find('div',attrs = {'class':'style__product-description___1vPQe'})
    if(name_table):
        for row in name_table:
            name += row.text
            name += ','
        name = name[:-1]
        cost_table = soup.find('div',attrs = {'class':'style__price-tag___B2csA'})

        for row in cost_table:
            cost += row.string

        return name + ": " + cost
    else :
        return medicine + ' not found'
    

In [43]:
def find_cost_pharmeasy(medicine):
    URL = "https://pharmeasy.in/search/all?name=" + medicine
    r = requests.get(URL)
    soup = BeautifulSoup(r.content,'html5lib')
    # print(soup.prettify())
    name = ""
    cost = ""
    desc = ""
    name_table = soup.find('h1',attrs = {'class':'ooufh'})
    if(name_table):
        for row in name_table:
            name = row.string
        name += ','    
        desc_table = soup.find('div',attrs = {'class':'_36aef'})
        if(desc_table):
            for row in desc_table:
                desc = row.string
            desc += ':'
            cost_table = soup.find('div',attrs = {'class':'_1_yM9'})
            for row in cost_table:
                cost +=  str(row.string)
            if cost[-1]=='*' :
                cost = cost[:-1]
            return name + desc + cost
        return medicine + ' not found'
    return medicine + ' not found'

In [96]:
from selenium.common.exceptions import NoSuchElementException

In [97]:
def find_cost_apollo(medicine):
    driver.get("https://www.apollopharmacy.in/tsearch?q="+medicine)
    try:
        unavailable = driver.find_element_by_class_name("no-products")
        return medicine + ' not found'
    except NoSuchElementException :
        pdt = driver.find_element_by_class_name("tagalys-product-tile")
        a = pdt.text.split('\n')
        return (a[0] + ' '+ a[1]).capitalize()

In [100]:
def find_cost_netmeds(medicine):
    URL = "https://www.netmeds.com/catalogsearch/result?q=" + medicine
    driver.get(URL)
    try:
        pdt = driver.find_element_by_class_name("drug_list")
        a = pdt.text.split('\n')[:5]
        return a[0] + a[2]
    except NoSuchElementException:
        return medicine + ' not found'

In [94]:
medicines = ['allegra','paracetamol','phenylephrine','sinarest','Atorvastatin' ,'Clopidogrel' ,'bdb']

In [104]:
for x in medicines:
    cost = find_cost_1mg(x)
    print("1mg: ",cost)
    cost = find_cost_pharmeasy(x)
    print("Pharmeasy: ",cost)
    print("Apollo: ",find_cost_apollo(x))
    print("Netmeds: ",find_cost_netmeds(x))

1mg:  Allegra-M Tablet ,strip of 10 tablets: MRP₹ 199.6
Pharmeasy:  Allegra 120mg Tablet,10 Tablet(s) in Strip:₹ 142.94
Apollo:  Allegra 120mg tablet ₹168.20
Netmeds:  Allegra 180mg Tablet 10'SRs.197.73
1mg:  Dolo 650 Tablet ,strip of 15 tablets: MRP₹ 30.74
Pharmeasy:  Facer Mr Tab,10 Tablet(s) in Strip:₹ 13.49
Apollo:  Crocin cold & flu tablet ₹54.90
Netmeds:  Crocin DS 240mg Suspension 60mlRs.50.61
1mg:  Solvin Decongestant Tablet ,strip of 15 tablets: MRP₹ 58.53
Pharmeasy:  Drosyn 10% Eye Drops 5ml,5ml Eye Drop in Bottle:₹ 29.96
Apollo:  Cheston cold tablet ₹39
Netmeds:  Sinarest AF Syrup 60mlRs.71.96
1mg:  Sinarest New Tablet ,strip of 10 tablets: MRP₹ 55.73
Pharmeasy:  sinarest not found
Apollo:  Sinarest tablet ₹50.70
Netmeds:  Sinarest New Tablet 10'SRs.50.71
1mg:  Aztor 10 Tablet ,strip of 15 tablets: MRP₹ 91.85
Pharmeasy:  Atorva 5mg Tab 15'S,15 Tablet(s) in Strip:₹ 82.28
Apollo:  Ecosprin av 75mg capsule ₹41.04
Netmeds:  Atorva 10Mg Tablet 15'SRs.91.89
1mg:  Clopitab Tablet ,